In [ ]:
[[360.77769585,   0.,         639.89330646],
 [  0.,         360.62443724, 359.0092385 ],
 [  0.,           0.,           1.        ]]
[[-0.00114422,  0.00073427,  0.00009126,  0.00001122, -0.00017835]]

In [58]:
import cv2.aruco as aruco
import cv2
import numpy as np
from itertools import combinations
np.set_printoptions(suppress=True)

dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters =  cv2.aruco.DetectorParameters()
# 
detector = aruco.ArucoDetector(dictionary, parameters)

In [2]:
board = aruco.CharucoBoard((8, 8), 0.4, 0.2, dictionary)

In [3]:
# show(board.generateImage((1024, 1024)))

In [111]:
img = cv2.imread("../machines/02_dobot/arucos.png")
if img.shape[-1] != 3:
    img_c = np.repeat(img[..., None], 3, axis=-1)
else:
    img_c = img.copy()

In [33]:
show(img)

In [10]:
images = [cv2.imread(f"../machines/02_dobot/charuco/charuco_{i}.png") for i in range(3)]
matched_points = []
corners_2 = []

for image in images:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cur_corners, cur_ids, _ = detector.detectMarkers(image)
    matched_points.append(list(
        map(np.squeeze, board.matchImagePoints(cur_corners, cur_ids))
    ))
    corners_2.append(aruco.interpolateCornersCharuco(
        cur_corners, cur_ids,
        img, board
    )[1:])
    
obj_points, img_points = zip(*matched_points)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, image.shape,
    None, None
)

In [122]:
img_c = aruco.drawDetectedCornersCharuco(img_c, corners2, ids2, [255, 0, 0])
show(img_c)

In [14]:
for i in range(3):
    img_c = images[i].copy()

    t = np.zeros((2, 3))
    _, rvec, tvec = aruco.estimatePoseCharucoBoard(
        *corners_2[i],
        board, mtx, dist,
        None, None
    )

    img_c = cv2.drawFrameAxes(img_c, mtx, dist, rvec, tvec, 0.4)

    show(img_c)

In [12]:
def show(img):
    cv2.imshow("window", img)
    while True:
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    cv2.destroyAllWindows()

In [69]:
cam = Camera()
cam.captured_images = [cv2.imread(f"../machines/02_dobot/charuco_1/charuco_{i}.png") for i in range(3)]

cam.calibrate()

In [29]:
show(cam.process_charuko(cam.captured_images[0])[0])

In [94]:
import pickle
with open(f"../machines/02_dobot/charuco_1/poses.pickle", "rb") as f:
    data = pickle.load(f)
base_to_end = data["base_to_end"]
cam_to_checker = data["cam_to_checker"]

In [112]:
def calc_normal(twists):
    return np.mean([twist[:3, 2] for twist in twists], axis=0)

def get_B(twist, n):
    R = twist[:3, :3]
    p = twist[:3, -1:]

    r_vec = n + [0, 0, 1]
    r_vec = r_vec / np.linalg.norm(r_vec) * np.pi
    R1 = cv2.Rodrigues(r_vec)[0]

    B = np.block([[R1, np.zeros((3, 1))], [0, 0, 0, 1]]) @ twist
    return B[np.ix_([0, 1, 3], [0, 1, 3])]

In [121]:
n

array([-0.17641429, -0.20707052,  0.9622887 ])

In [119]:
As[1] @ get_X(*optim.x) @ Bs[1]

array([[-0.99999994,  0.00020325,  0.20016782],
       [ 0.00020332,  0.99999995,  0.09994503],
       [ 0.        ,  0.        ,  1.        ]])

In [114]:
from itertools import combinations
def get_X(phi, dx, dy):
    return np.array([
        [np.cos(phi), -np.sin(phi), dx],
        [np.sin(phi), np.cos(phi), dy],
        [0, 0, 1]
    ])

def solver(x):
    X = get_X(*x)
    l = left @ X
    r = np.vstack([(X @ right)[:2] for right in rights])
    loss = np.linalg.norm(l - r, "fro")
    return loss

In [115]:
import scipy.optimize as optimize
optim = optimize.minimize(solver, [0, 0, 0], options=dict(maxiter=10_000))

In [116]:
optim

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.0005930198346456394
        x: [ 1.490e+00 -4.241e-04  5.008e-02]
      nit: 21
      jac: [-3.682e-07  3.185e-06 -2.305e-06]
 hess_inv: [[ 5.289e-02 -1.619e-03 -1.302e-02]
            [-1.619e-03  1.902e-04  3.977e-04]
            [-1.302e-02  3.977e-04  3.345e-03]]
     nfev: 136
     njev: 34

In [92]:
left @ get_X(*optim.x) - np.vstack([(get_X(*optim.x) @ right)[:2] for right in rights])

array([[ 0.00024252,  0.00021116, -0.00030856],
       [-0.00021116,  0.00024252,  0.00019344],
       [ 0.00027842, -0.000139  , -0.00027269],
       [ 0.000139  ,  0.00027842, -0.00015935],
       [ 0.00018894, -0.00015969, -0.00023699],
       [ 0.00015969,  0.00018894, -0.00016928],
       [-0.00000472,  0.00000924,  0.00002517],
       [-0.00000924, -0.00000472,  0.00005281],
       [-0.00001715,  0.00007217,  0.00000932],
       [-0.00007217, -0.00001715,  0.00011352],
       [-0.00006301,  0.00001004, -0.00001444],
       [-0.00001004, -0.00006301,  0.00004286]])

array([[ 0.65654461, -0.7542872 ,  0.09387064],
       [ 0.7542872 ,  0.65654461,  0.11354144],
       [-0.44682104, -0.89462336,  0.09109844],
       [ 0.89462336, -0.44682104, -0.09853438],
       [-0.64560109, -0.76367482,  0.13103933],
       [ 0.76367482, -0.64560109, -0.13444103],
       [-0.89065726, -0.45467531,  0.15988141],
       [ 0.45467531, -0.89065726, -0.22892253],
       [-0.97288419, -0.23129277,  0.22869008],
       [ 0.23129277, -0.97288419, -0.24487879],
       [-0.15729674, -0.98755138,  0.01877084],
       [ 0.98755138, -0.15729674, -0.0601117 ]])

In [99]:
B

array([[ 0.77064702,  0.63726225, -0.08513299],
       [-0.63726225,  0.77064702, -0.28344694],
       [ 0.        ,  0.        ,  1.        ]])

In [85]:
cv2.calibrateHandEye(
    *zip(*base_to_end),
    *zip(*cam_to_checker),
    method=cv2.CALIB_HAND_EYE_HORAUD
)

(array([[ 0.06304504,  0.99670787,  0.05097785],
        [ 0.10538187,  0.04414555, -0.99345147],
        [-0.99243135,  0.06800433, -0.10225178]]),
 array([[-3.88976162e+00],
        [ 1.23383242e+10],
        [ 1.21889556e+01]]))

In [30]:
import pybullet as pb
from PIL import Image as im
import io
import cv2
import numpy as np
import cv2.aruco as aruco
import os

class Camera:
    QUALITY = 80

    default_size = {
        'width': 1280,
        'height': 720
    }

    def __init__(self, size=None):
        if size is None:
            size = Camera.default_size
        self.size = size
        self.viewMatrix = None
        
        self.projectionMatrix = pb.computeProjectionMatrixFOV(
            fov=90.0,
            aspect=1280 / 720,
            nearVal=0.1,
            farVal=100
        )
        self.captured_images = []
        
        self.dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
        self.parameters =  cv2.aruco.DetectorParameters()
        self.detector = aruco.ArucoDetector(self.dictionary, self.parameters)
        self.tile_size = 2 * .2 / 8
        self.board = aruco.CharucoBoard((8, 8), self.tile_size, self.tile_size / 2, self.dictionary)
        
        self.mtx, self.dist = None, None

    def set_pos(self, pos, up_vector, direction):
        pos = np.array(pos)
        up = np.cross(np.cross(direction, up_vector), direction)
        
        self.viewMatrix = pb.computeViewMatrix(
            cameraEyePosition=pos,
            cameraTargetPosition=pos + direction,
            cameraUpVector=up_vector
        )

    def get_frame(self, save=True) -> np.ndarray:
        """
        returns RGBA array of size (x, y, 4)
        """
        if self.viewMatrix is None:
            return np.array([])
        params = {
            **self.size,
            'viewMatrix': self.viewMatrix,
            'projectionMatrix': self.projectionMatrix,
            'renderer': pb.ER_BULLET_HARDWARE_OPENGL
        }
        frame = pb.getCameraImage(**params)[2][:, :, :3].astype(np.uint8)
        if save:
            self.captured_images.append(frame)
        return frame
    
    def calibrate(self):
        matched_points = []
        for image in self.captured_images:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cur_corners, cur_ids, _ = self.detector.detectMarkers(image)
            matched_points.append(list(
                map(np.squeeze, self.board.matchImagePoints(cur_corners, cur_ids))
            ))
        obj_points, img_points = zip(*matched_points)

        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
            obj_points, img_points, image.shape,
            None, None
        )
        self.mtx = mtx
        self.dist = dist
    
    def save(self, folder):
        for i, frame in enumerate(self.captured_images):
            if not os.path.exists(f"./{folder}/"):
                os.mkdir(f"./{folder}/")
            cv2.imwrite(f"{folder}/charuco_{i}.png", frame)
    
    def process_charuko(self, frame):
        assert self.mtx is not None and self.dist is not None
        
        frame = frame.copy()
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        corners, ids, _ = self.detector.detectMarkers(gray)
        
        if ids is None:
            cv2.putText(frame, "No Ids", (0,64), font, 1, (0,255,0),2,cv2.LINE_AA)
            return frame
        
        _, corners_checker, ids_checker = aruco.interpolateCornersCharuco(
            corners, ids,
            gray, self.board
        )
        
        _, rvec_check, tvec_check = aruco.estimatePoseCharucoBoard(
            corners_checker, ids_checker, 
            self.board, self.mtx, self.dist,
            None, None
        )
        
        cv2.drawFrameAxes(frame, self.mtx, self.dist, rvec_check, tvec_check, 0.1)
        aruco.drawDetectedMarkers(frame, corners)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, self.tile_size / 2, self.mtx, self.dist)
        
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.drawFrameAxes(frame, self.mtx, self.dist, rvec, tvec, 0.03)
            
        return frame, rvec_check, tvec_check
